# <font color="orange"> House Prices - Advanced Regression Techniques </font>

#### Predict sales prices and practice feature engineering, RFs, and gradient boosting

In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots

In [5]:
DataFolder = "/Users/manideepbangaru/Documents/EDAnMLApply/Datasets/house-prices-advanced-regression-techniques/"
OutputFolder = "/Users/manideepbangaru/Documents/EDAnMLApply/Output"

In [6]:
hdf = pd.read_csv(DataFolder+"train.csv",index_col=0)

In [7]:
hdf.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
for cols in hdf.columns:
    nullsum = round((hdf[cols].isnull().sum()/len(hdf))*100,2)
    if nullsum >= 50 :
        hdf.drop(cols,axis=1,inplace=True)
        print("dropped column %s which has %s percentage of null values"%(cols,nullsum))

dropped column Alley which has 93.77 percentage of null values
dropped column PoolQC which has 99.52 percentage of null values
dropped column Fence which has 80.75 percentage of null values
dropped column MiscFeature which has 96.3 percentage of null values


In [9]:
fig = px.box(hdf["MSSubClass"])

In [10]:
px.scatter(hdf["MSSubClass"])

In [11]:
fig = make_subplots(rows=1, cols = 2)
fig.add_trace(
    go.Histogram(x =  hdf["MSSubClass"]),row=1,col=1)
fig.add_trace(
    go.Box(x = hdf["MSSubClass"]),row=1,col=2)
fig.show()

In [ ]:
import seaborn as sns